In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file, output_server
import datetime as dt

In [2]:
data = pd.read_csv('../data/train.csv')
data.head()

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [3]:
data['DateTime'] = pd.to_datetime(data['Date'])
data['Year'] = data['DateTime'].apply(lambda x: x.year)
data['Month'] = data['DateTime'].apply(lambda x: x.month)
data['Day'] = data['DateTime'].apply(lambda x: x.day)
data['WeekNumber'] = data['DateTime'].apply(lambda x: x.isocalendar()[1])

In [4]:
from sklearn.preprocessing import LabelEncoder
data.ix[:, ['StateHoliday']] = data.ix[:, ['StateHoliday']].astype(str)

data['StateHoliday'] = LabelEncoder().fit_transform(data['StateHoliday'])


In [6]:
to_predict = data.ix[data['DateTime'] > dt.datetime(2015, 5, 1), :]
old_data = data.ix[data['DateTime'] < dt.datetime(2015, 5, 1), :]

In [7]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(old_data.drop(['Sales', 'Date', 'DateTime'], axis=1), 
                                                    old_data['Sales'], 
                                                    test_size=0.25, 
                                                    random_state=8675309)

In [8]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

AB = AdaBoostRegressor(DecisionTreeRegressor(),
                       n_estimators = 100, 
                       random_state=8675309).fit(X_train, y_train)

In [9]:
to_predict.ix[:, 'predictedSales'] = AB.predict(to_predict.drop(['Sales', 'Date', 'DateTime'], axis=1))

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
to_predict.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,DateTime,Year,Month,Day,WeekNumber,predictedSales
0,1,5,2015-07-31,5263,555,1,1,0,1,2015-07-31,2015,7,31,31,5274
1,2,5,2015-07-31,6064,625,1,1,0,1,2015-07-31,2015,7,31,31,6102
2,3,5,2015-07-31,8314,821,1,1,0,1,2015-07-31,2015,7,31,31,8898
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015-07-31,2015,7,31,31,12533
4,5,5,2015-07-31,4822,559,1,1,0,1,2015-07-31,2015,7,31,31,5267


In [13]:
from sklearn.metrics import mean_squared_error
from math import sqrt

sqrt(mean_squared_error(to_predict['Sales'], to_predict['predictedSales']))

849.6108048837962

In [14]:
datetime_groups = old_data.ix[data['Open']==1, :].groupby('DateTime')

dates = []
sales = []
for x, y in datetime_groups:
    dates.append(x)
    sales.append(y['Sales'].mean())

In [15]:
pred_datetime_groups = to_predict.ix[data['Open']==1, :].groupby('DateTime')

pred_dates = []
pred_sales = []
actual_sales = []
for x, y in pred_datetime_groups:
    pred_dates.append(x)
    pred_sales.append(y['predictedSales'].mean())
    actual_sales.append(y['Sales'].mean())

In [57]:
output_server("Projected Sales")

p = figure(x_axis_type = "datetime", 
           y_axis_type="log",  
           plot_width=1000, 
           plot_height=700, 
           title="Projected Sales")
p.xaxis.axis_label="Date"
p.yaxis.axis_label="Sales (log scale)"

p.line(dates, sales , line_width=2, color='#1F78B4')
p.line(pred_dates, pred_sales, color='#FB9A99')
# p.line(pred_dates[:50], actual_sales[:50], color='#01DF01')


show(p)



Using saved session configuration for http://localhost:5006/
To override, pass 'load_from_config=False' to Session


In [68]:
graph_data = {"Algorithm": ["Decision Tree", "Neural Net", "Linear SVR", "Adaboosted Tree", "Gradient Boost"],
              "RMSE": [1115.85, 3865.83, 1540.97, 789.80, 1540.9]
             }



# graph_data = {"x": [1115.85, 3865.83, 1540.97, 789.80, 1540.9]}
graph_cats = ["Decision Tree", "Neural Net", "Linear SVR", "Adaboosted Tree", "Gradient Boost"]

df = pd.DataFrame(graph_data)
df


,Algorithm,RMSE
0,Decision Tree,1115.85
1,Neural Net,3865.83
2,Linear SVR,1540.97
3,Adaboosted Tree,789.80
4,Gradient Boost,1540.90


In [69]:
from bokeh.charts import Bar, output_file

b = Bar(df, label='Algorithm', values="RMSE", title='RMSE of Algorithms', xlabel='Algorithm', ylabel='RMSE')

output_file("bar.html")

show(b)

ERROR:/usr/local/lib/python2.7/dist-packages/bokeh/validation/check.pyc:W-1001 (NO_GLYPH_RENDERERS): Plot has no glyph renderers: Figure, ViewModel:Plot, ref _id: c6f2476b-ffc4-4f6f-bb65-34eca3368553


In [75]:
full_dates_grouped = data.groupby('DateTime')

promo_dates = []
promo_1_sales = []
promo_2_sales = []

for x, y in full_dates_grouped:
    promo_dates.append(x)
    promo_1_sales.append[y['Sales'].mean()]
    promo_0_sales.append[y[y['Promo']!=1,'Sales'].mean()]

TypeError: 'builtin_function_or_method' object has no attribute '__getitem__'